# Исследование надежности заемщиков


Исследование надёжности заёмщиков
Заказчик — кредитный отдел банка.

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.



## Откройте таблицу и изучите общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные в датасете содержат 12 столбцов и 21525 строк. 7 столбцов содержат количественные переменные, 5 столбцов - категориаальные переменные. В столбцах 'total_income', 'days_employed', 'gender' встречаются пропуски в данных.

В столбце 'gender' встречается значение 'XNA', которое, вероятно, связано с тем, что заемщик при заполнении анкеты не указал свой пол.

Также в столбце 'days_employed' встречаются отрицательные значения. И сами значения, если это действительно, дни - нереальны (стаж более чем в 100 лет и т.д.). Ошибка в форматировании данных? Стаж в днях указан в Unix time?

В столбце 'children' встречаются отрицательные значения (количество детей =-1).
В столбце 'dob_years' встречаются нули (возраст равен 0). Возможно, потому, что заемщик не указал возраст при заполнении анкеты.

Значения в столбцах 'purpose', 'education' дублируются (текстовые переменные).

## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 На сумму дохода сильнее всего влияет тип занятости, поэтому  пропуски в этом столбце заполним медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()


### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Выведем перечень уникальных значений столбца `children`

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения, которые необходимо удалить.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. 

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.




In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.



In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data['debt'].unique() # Убедимся, что в столбце 'debt' значения бывают только 0 и 1.

array([0, 1])

In [23]:
data_debt = data.groupby('children')['debt'].value_counts() # Определим в абсолютном выражении количество людей с долгами и без в зависимости от количества детей.
print(data_debt)

children  debt
0         0       13028
          1        1063
1         0        4364
          1         444
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64


In [24]:
data3 = data['children'].loc[(data['children'] == 0) & (data['debt'] == 1)].value_counts() # Убедимся в количестве людей не имеющих детей и имеющих долг (для последующей проверке применения метода sum())

print(data3)


0    1063
Name: children, dtype: int64


In [25]:
data4 = data['children'].loc[(data['children'] == 0) & (data['debt'] == 0)].value_counts() # Убедимся в количестве людей не имеющих детей и yt имеющих долг (для последующей проверке применения метода count())

print(data4)

0    13028
Name: children, dtype: int64


In [26]:
debt_from_children = pd.DataFrame() # Создадим новую таблицу
debt_from_children['quantity'] = data.groupby('children')['debt'].count() # посчитаем общее количество клиентов разнесенное в зависисмости от количества детей
debt_from_children['quantity_debt'] = data.groupby('children')['debt'].sum() # посчитаем количество просрочек в зависисомсти от количества детей
debt_from_children['percentage'] = debt_from_children['quantity_debt'] / debt_from_children['quantity'] # найдем процентное соотношение этих величин
print(debt_from_children)

          quantity  quantity_debt  percentage
children                                     
0            14091           1063    0.075438
1             4808            444    0.092346
2             2052            194    0.094542
3              330             27    0.081818
4               41              4    0.097561
5                9              0    0.000000


In [27]:
debt_from_children.sort_values('percentage', ascending = False) # Отсортируем процент просрочек в заисимости от количества детей по убыванию

,quantity,quantity_debt,percentage
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4808,444,0.092346
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


In [28]:

data.groupby('children')['debt'].agg(['count', 'sum', 'mean'])\
    .sort_values(by = 'mean', ascending = False)

,count,sum,mean
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4808,444,0.092346
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


In [29]:
debt_from_children.loc[1:4, 'percentage'].mean() # Посчитаем средний процент просрочек при наличии детей (5 детей исключено из-за малой репрезентативности)

0.09156678940239288

**Вывод:** Наличие детей повышает вероятность невыплаты кредита людьми, имеющими детей в среднем на 1,5%, по отношению к бездетным.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
family_debt = data.groupby('family_status')['debt'].value_counts() # Определим в абсолютном выражении количество людей с долгами и без в зависимости от семейного положения.

In [31]:
debt_family = pd.DataFrame() # Создадим новую таблицу
debt_family['quantity'] = data.groupby('family_status')['debt'].count() # посчитаем общее количество клиентов разнесенное в зависисмости от семейного положения.
debt_family['quantity_debt'] = data.groupby('family_status')['debt'].sum() # посчитаем количество просрочек в зависисомсти от от семейного положения.
debt_family['percentage'] = debt_family['quantity_debt'] / debt_family['quantity'] # найдем процентное соотношение этих величин
print(debt_family)

                       quantity  quantity_debt  percentage
family_status                                             
Не женат / не замужем      2796            273    0.097639
в разводе                  1189             84    0.070648
вдовец / вдова              951             63    0.066246
гражданский брак           4134            385    0.093130
женат / замужем           12261            927    0.075606


In [32]:
debt_family.sort_values('percentage', ascending = False) # Отсортируем значения по убыванию

,quantity,quantity_debt,percentage
family_status,,,
Не женат / не замужем,2796,273,0.097639
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


In [33]:
debt_family = debt_family.reset_index(drop=True)
print(debt_family)



   quantity  quantity_debt  percentage
0      2796            273    0.097639
1      1189             84    0.070648
2       951             63    0.066246
3      4134            385    0.093130
4     12261            927    0.075606


In [34]:
debt_family.loc[0:3, 'percentage'].mean() # Посчитаем средний процент просрочек при отсутсвии брака.

0.0819158212721453

In [35]:
debt_family.loc[4, 'percentage'].mean() # Посчитаем средний процент просрочек в бракe.

0.07560557866405676

In [36]:
debt_family.loc[2:4, 'percentage'].mean() # Посчитаем средний процент просрочек побывавших в бракe.

0.07832725858211426

**Вывод:** Вероятность невозврата кредита для людей не состящих в браке выше, чем у семейных клиентов. Кроме того, люди состоявшие в браке (+ вдовцы и разведенные) имеют меньше просрочек, относительно не женатых.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
data_income = data.groupby('total_income_category')['debt'].value_counts() 
print(data_income)

total_income_category  debt
A                      0          23
                       1           2
B                      0        4660
                       1         354
C                      0       14568
                       1        1353
D                      0         328
                       1          21
E                      0          20
                       1           2
Name: debt, dtype: int64


In [38]:
debt_income = pd.DataFrame() # Создадим новую таблицу
debt_income['quantity'] = data.groupby('total_income_category')['debt'].count() # посчитаем общее количество клиентов разнесенное в зависисмости от категории дохода.
debt_income['quantity_debt'] = data.groupby('total_income_category')['debt'].sum() # посчитаем количество просрочек в зависисомсти от категории дохода.
debt_income['percentage'] = debt_income['quantity_debt'] / debt_income['quantity'] # найдем процентное соотношение этих величин
print(debt_income)

                       quantity  quantity_debt  percentage
total_income_category                                     
A                            25              2    0.080000
B                          5014            354    0.070602
C                         15921           1353    0.084982
D                           349             21    0.060172
E                            22              2    0.090909


In [39]:
debt_income.sort_values('percentage', ascending = False) # Отсортируем значения по убыванию

,quantity,quantity_debt,percentage
total_income_category,,,
E,22,2,0.090909
C,15921,1353,0.084982
A,25,2,0.080000
B,5014,354,0.070602
D,349,21,0.060172


**Вывод:** Прямой связи между уровнем дохода и вероятностью вернуть кредит не прослеживается. Категории A, E, D обладают малой выборкой и не являются репрезентативными.
Можно еще сделать вывод, что люди имеющие доход от 50000 до 200000 самые многочисленные заемщики. Это может свидетельствовать о том, что данный диапазон является основным, то есть к нему относится большинство клиентов (так называемый средний класс). Люди получающие больше уже могут быть отнесены к богатым. Соответственно, можно сделать вывод, что вероятность вернуть кредит в срок у богатых людей больше.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [40]:
data_purpose = data.groupby('purpose_category')['debt'].value_counts() 
print(data_purpose) # Рассмотрим величин .

purpose_category          debt
операции с автомобилем    0       3879
                          1        400
операции с недвижимостью  0       9971
                          1        780
получение образования     0       3619
                          1        369
проведение свадьбы        0       2130
                          1        183
Name: debt, dtype: int64


In [41]:
debt_purpose = pd.DataFrame() # Создадим новую таблицу
debt_purpose['quantity'] = data.groupby('purpose_category')['debt'].count() # посчитаем общее количество клиентов разнесенное в зависисмости от цели кредита.
debt_purpose['quantity_debt'] = data.groupby('purpose_category')['debt'].sum() # посчитаем количество просрочек в зависисомсти от цели кредита.
debt_purpose['percentage'] = debt_purpose['quantity_debt'] / debt_purpose['quantity'] # найдем процентное соотношение этих величин
display(debt_purpose)
debt_purpose

,quantity,quantity_debt,percentage
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


,quantity,quantity_debt,percentage
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** Кредиты, связанные с операциями с автомобилям и с получением образования имеют большую вероятность просрочки.

### Шаг 4: общий вывод.

Исследование надежности заемщиков было основано на оценке массива информации, содержащего данные о поле, семейном положении, возрасте, уровне дохода и просрочках по кредиту заемщиков. 
На первоначальном этапе была проведена предобработка данных, которая установила наличие пропусков в данных о трудовом стаже и доходах клиентов. Эти пропуски были заменены на медианные показатели с учетом типа занятости клиентов.
Далее были устранены аномалии, связанные с наличием отрицательного стажа и количества детей. Также перед проведением оценки соответствющие показатели переведены в количественные, данные приведены к единому регистру и удалены дубликаты в данных. Проведена категоризация клиентов по степени дохода и по цели кредита.

Проведя исследование надежности заемщиков было установлено: 

- наличие детей повышает вероятность невыплаты кредита людьми, имеющими детей в среднем на 1,5%, по отношению к бездетным;

- вероятность невозврата кредита для людей не состящих в браке выше, чем у семейных клиентов. При этом, люди находившиеся когда-либо в официальном браке вернут кредит с большей вероятностью;

- вероятность возврата кредита зависит от цели, для которой он оформляется;

- четкой корреляции между уровнем дохода клиента и вероятностью возврата кредита не установлено. При этом, существует вероятность, что люди среднего достатка будут иметь просрочки по кредиту, относительно богатых. 

Рекомендации: 

- при сборе информации о клиентах, сотрудникам проверять заполнение всех граф анкеты о клиенте;

- провести проверку ввода, обработки и хранения информации в системе на наличие ошибок, приводящих к потере информации.